# Bitcoin Thresholds Analysis

This notebook explores the profitability thresholds for various mining strategies in Bitcoin, including:
- **Section 3.1**: Orphan block mining threshold
- **Section 3.2**: Rational withholding threshold (zero connectivity)

## Theoretical Background

In Bitcoin mining, rational miners may deviate from honest behavior if it increases their expected rewards. Two key questions arise:

1. **When should a miner mine on their own orphan block** (one block behind the main chain)?
2. **When should a miner withhold a newly discovered block** instead of broadcasting it immediately?

The answer depends on the miner's **hashing power (q)** and **network connectivity (γ)**.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

---

## Section 3.1: Orphan Block Mining Threshold

### Problem Statement

An otherwise honest miner discovers a block but realizes another miner has already published a competing block at the same height. Should the miner:
- **Abandon their block** and mine on top of the competitor's block (honest strategy)?
- **Continue mining** on their own block, hoping to catch up (deviant strategy 1+2)?

### Strategy 1+2

This strategy consists of:
1. **Strategy 1**: If one block behind, continue mining on your own chain
2. **Strategy 2**: If you catch up (tie at same height), continue mining on your chain

The expected yield for strategy 1+2 with zero connectivity (γ=0) is:

$$T_{1+2}(q) = \frac{q^2(4-q)}{1 + q + q^3}$$

where **q** is the miner's hashing power fraction.

### Threshold Calculation

The miner should deviate when:
$$T_{1+2}(q) > q$$

Solving $T_{1+2}(q) = q$ gives us the threshold:
$$q^* = \sqrt{2} - 1 \approx 0.4142$$

In [ ]:
# Calculate the theoretical threshold
threshold_orphan = np.sqrt(2) - 1

print("="*60)
print("SECTION 3.1: ORPHAN BLOCK MINING THRESHOLD")
print("="*60)
print(f"\nTheoretical threshold: q* = √2 - 1 = {threshold_orphan:.6f}")
print(f"Percentage: {threshold_orphan*100:.2f}% of network hashing power")
print(f"\nInterpretation:")
print(f"- If q < {threshold_orphan:.4f}: Mine honestly (abandon orphan block)")
print(f"- If q > {threshold_orphan:.4f}: Continue mining on orphan block (deviant)")
print("="*60)

In [ ]:
# Define the yield function for strategy 1+2
def strategy_1_2_yield(q):
    """
    Calculate the expected yield for strategy 1+2 with zero connectivity.
    
    Parameters:
    q (float or array): Miner's hashing power fraction (0 < q < 1)
    
    Returns:
    float or array: Expected yield T(q)
    """
    return (q**2 * (4 - q)) / (1 + q + q**3)

# Verify with a test value
q_test = 0.415
yield_test = strategy_1_2_yield(q_test)

print(f"\n--- Verification ---")
print(f"For q = {q_test}:")
print(f"  Yield T(q) = {yield_test:.6f}")
print(f"  Honest yield = {q_test:.6f}")
print(f"  Gain from deviation = {(yield_test - q_test)*100:.4f}%")

if yield_test > q_test:
    print(f"  ✓ Strategy 1+2 is MORE profitable than honest mining")
else:
    print(f"  ✗ Honest mining is more profitable")

In [ ]:
# Visualization: Yield curves
q_range = np.linspace(0.01, 0.6, 1000)
yield_1_2 = strategy_1_2_yield(q_range)
honest_yield = q_range

plt.figure(figsize=(14, 7))

# Plot yields
plt.plot(q_range, honest_yield, 'b-', linewidth=2, label='Honest Mining (yield = q)', alpha=0.8)
plt.plot(q_range, yield_1_2, 'r-', linewidth=2, label='Strategy 1+2 (deviant)', alpha=0.8)

# Mark the threshold
plt.axvline(x=threshold_orphan, color='green', linestyle='--', linewidth=2, 
            label=f'Threshold q* = √2-1 ≈ {threshold_orphan:.4f}', alpha=0.7)

# Fill regions
mask_honest = q_range < threshold_orphan
mask_deviant = q_range >= threshold_orphan

plt.fill_between(q_range[mask_honest], 0, 1, alpha=0.1, color='blue', 
                  label='Honest mining optimal')
plt.fill_between(q_range[mask_deviant], 0, 1, alpha=0.1, color='red', 
                  label='Deviant strategy optimal')

# Styling
plt.xlabel('Hashing Power (q)', fontsize=13, fontweight='bold')
plt.ylabel('Expected Yield', fontsize=13, fontweight='bold')
plt.title('Section 3.1: Orphan Block Mining - Profitability Analysis', 
          fontsize=15, fontweight='bold', pad=20)
plt.legend(fontsize=11, loc='upper left')
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.6)
plt.ylim(0, 0.5)

# Add annotations
plt.annotate(f'q* = {threshold_orphan:.4f}\n({threshold_orphan*100:.2f}%)', 
             xy=(threshold_orphan, strategy_1_2_yield(threshold_orphan)), 
             xytext=(threshold_orphan + 0.08, 0.35),
             fontsize=12, fontweight='bold',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.3', lw=2))

plt.tight_layout()
plt.show()

print(f"\n📊 Graph shows that Strategy 1+2 becomes profitable above q* = {threshold_orphan:.4f}")

### Key Findings (Section 3.1)

1. **Threshold**: A miner with more than **41.42%** of the network's hashing power should mine on their orphan block
2. **Economic Impact**: This creates a centralization pressure - large miners have more profitable strategies
3. **Network Security**: Below this threshold, honest mining remains the Nash equilibrium

---

## Section 3.2: Rational Withholding Threshold (γ = 0)

### Problem Statement

A rational miner with **zero connectivity** (γ = 0) discovers a new block on top of the main chain. Should they:
- **Broadcast immediately** (honest behavior)?
- **Withhold the block** and continue mining to build a private chain (selfish behavior)?

With zero connectivity, if the miner withholds their block, honest miners will compete on the previous block. This creates a situation similar to the orphan block problem.

### Analysis

The withholding strategy is equivalent to **intentionally creating an orphan situation** for honest miners. The profitability condition is the same:

$$T_{1+2}(q) > q$$

Therefore, the threshold is again:
$$q^* = \sqrt{2} - 1 \approx 0.4142$$

We verify this numerically by solving:
$$T_{1+2}(q) - q = 0$$

In [ ]:
# Define the objective function to find the threshold
def objective_function(q):
    """
    Objective function for finding the threshold.
    We want to find q such that T(q) - q = 0
    
    Parameters:
    q (float): Hashing power fraction
    
    Returns:
    float: T(q) - q
    """
    return strategy_1_2_yield(q) - q

# Solve numerically using scipy.optimize.fsolve
# Initial guess: 0.4 (we know the solution is around this value)
q_threshold = fsolve(objective_function, 0.4)[0]

print("="*60)
print("SECTION 3.2: RATIONAL WITHHOLDING THRESHOLD (γ=0)")
print("="*60)
print(f"\nNumerically computed threshold: q* = {q_threshold:.6f}")
print(f"Percentage: {q_threshold*100:.2f}% of network hashing power")
print(f"\nTheoretical value: √2 - 1 = {np.sqrt(2)-1:.6f}")
print(f"Difference: {abs(q_threshold - (np.sqrt(2)-1)):.2e}")
print("="*60)

In [ ]:
# Verification of the solution
yield_at_threshold = strategy_1_2_yield(q_threshold)

print(f"\n--- Verification ---")
print(f"At threshold q* = {q_threshold:.6f}:")
print(f"  Yield T(q*) = {yield_at_threshold:.6f}")
print(f"  Honest yield q* = {q_threshold:.6f}")
print(f"  Difference |T(q*) - q*| = {abs(yield_at_threshold - q_threshold):.2e}")

if abs(yield_at_threshold - q_threshold) < 1e-6:
    print(f"  ✓ Solution verified: T(q*) ≈ q*")
    
print(f"\n--- Economic Interpretation ---")
print(f"For q < {q_threshold:.4f}: Broadcast blocks immediately (honest mining)")
print(f"For q > {q_threshold:.4f}: Consider withholding blocks (selfish mining)")
print(f"\nNote: This assumes zero connectivity (γ=0)")

In [ ]:
# Visualization: Profit difference
q_range_fine = np.linspace(0.3, 0.5, 1000)
profit_difference = strategy_1_2_yield(q_range_fine) - q_range_fine

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Profit difference
ax1.plot(q_range_fine, profit_difference, 'purple', linewidth=3, label='T(q) - q')
ax1.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
ax1.axvline(x=q_threshold, color='red', linestyle='--', linewidth=2, 
            label=f'Threshold q* = {q_threshold:.4f}', alpha=0.8)

# Fill regions
mask_negative = profit_difference < 0
mask_positive = profit_difference >= 0

ax1.fill_between(q_range_fine[mask_negative], profit_difference[mask_negative], 0, 
                 alpha=0.3, color='blue', label='Honest mining better')
ax1.fill_between(q_range_fine[mask_positive], profit_difference[mask_positive], 0, 
                 alpha=0.3, color='red', label='Withholding better')

ax1.set_xlabel('Hashing Power (q)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Profit Difference (T(q) - q)', fontsize=12, fontweight='bold')
ax1.set_title('Profitability of Block Withholding', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Right plot: Relative gain
relative_gain = (strategy_1_2_yield(q_range_fine) / q_range_fine - 1) * 100

ax2.plot(q_range_fine, relative_gain, 'darkgreen', linewidth=3)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=1, alpha=0.5)
ax2.axvline(x=q_threshold, color='red', linestyle='--', linewidth=2, 
            label=f'Threshold q* = {q_threshold:.4f}', alpha=0.8)

# Shade regions
ax2.fill_between(q_range_fine, 0, relative_gain, 
                 where=(relative_gain < 0), alpha=0.3, color='blue',
                 label='Honest mining better')
ax2.fill_between(q_range_fine, 0, relative_gain, 
                 where=(relative_gain >= 0), alpha=0.3, color='red',
                 label='Withholding better')

ax2.set_xlabel('Hashing Power (q)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Relative Gain (%)', fontsize=12, fontweight='bold')
ax2.set_title('Percentage Gain from Withholding', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.suptitle('Section 3.2: Rational Withholding Analysis (γ=0)', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"\n📊 Graphs show profitability crossover at q* = {q_threshold:.4f}")

### Key Findings (Section 3.2)

1. **Threshold Confirmation**: With zero connectivity (γ=0), the withholding threshold is **q* = √2 - 1 ≈ 0.4142**
2. **Same as Orphan Mining**: The threshold is identical to Section 3.1, confirming the theoretical equivalence
3. **Connectivity Matters**: This analysis assumes γ=0. Higher connectivity changes the optimal strategy
4. **Network Vulnerability**: Miners with >41.42% of hash power have incentive to deviate from honest mining

---

## Comparative Analysis: Both Thresholds

In [ ]:
# Summary comparison
print("="*70)
print("SUMMARY: BITCOIN THRESHOLD ANALYSIS")
print("="*70)
print(f"\n{'Strategy':<30} {'Threshold (q*)':<20} {'Percentage'}")
print("-"*70)
print(f"{'3.1 Orphan Block Mining':<30} {threshold_orphan:<20.6f} {threshold_orphan*100:.2f}%")
print(f"{'3.2 Block Withholding (γ=0)':<30} {q_threshold:<20.6f} {q_threshold*100:.2f}%")
print("-"*70)
print(f"{'Theoretical Value: √2 - 1':<30} {np.sqrt(2)-1:<20.6f} {(np.sqrt(2)-1)*100:.2f}%")
print("="*70)

print(f"\n🔍 Key Insight:")
print(f"Both strategies become profitable at the SAME threshold: q* ≈ {threshold_orphan:.4f}")
print(f"This represents a fundamental vulnerability in Bitcoin's consensus mechanism.")
print(f"\n⚠️  Security Implication:")
print(f"The network is secure as long as no single miner controls >41.42% of hash power.")

## Implications for Bitcoin Security

### Economic Centralization Pressure

The existence of these thresholds creates economic incentives for centralization:
- Small miners (q < 0.414) must always mine honestly
- Large miners (q > 0.414) have profitable deviation strategies
- This creates a "rich get richer" dynamic

### Defense Mechanisms

1. **Mining Pool Decentralization**: Encouraging multiple competing pools
2. **Network Connectivity**: High γ values make selfish strategies less profitable
3. **Protocol Modifications**: Alternative consensus mechanisms (e.g., Ethereum's Proof-of-Stake)
4. **Community Vigilance**: Monitoring hash power distribution

### Practical Considerations

- **Current State**: Bitcoin's largest mining pools have <20% of hash power each
- **Safety Margin**: Well below the 41.42% threshold
- **Dynamic Nature**: Hash power distribution changes over time
- **Coordination**: Multiple small miners coordinating could approach the threshold

---

## Conclusion

This analysis reveals a fundamental trade-off in Bitcoin's security model:
- The network is **secure** as long as no miner exceeds ~41% of hash power
- Above this threshold, rational economic behavior diverges from honest mining
- Both orphan block mining and block withholding share the same profitability threshold
- The theoretical threshold **√2 - 1 ≈ 0.4142** is precisely confirmed numerically

These findings underscore the importance of maintaining a decentralized mining ecosystem for Bitcoin's long-term security.